<a href="https://colab.research.google.com/github/saeid-6780/Big-Data-PRJ/blob/main/STEP7_LogReg_PYSpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# for colab only
!pip install pyspark --quiet
from google.colab import drive
drive.mount('/content/gdrive')


     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 14.6 MB/s 
Mounted at /content/gdrive


In [2]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext()
sc.version

'3.3.0'

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-hw').getOrCreate()
df = spark.read.csv('/content/gdrive/MyDrive/BD/data.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- Date/Time: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Lon: double (nullable = true)
 |-- Base: string (nullable = true)



In [4]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql import functions as F
df=df.withColumn("id",F.monotonically_increasing_id())
df=df.withColumnRenamed("DATE/time","datetime")

func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M:%S'), TimestampType())
df2 = df.withColumn('timestamp', func(col('datetime')))
#df.withColumn("timestamp",to_timestamp(lit('06-24-2019 Date/Time'),'MM/dd/yyyy H:mm:ss')).show(truncate=False)
df2.show()
#df=df.withColumnRenamed("DATE/time","datetime")
#df.show()
#df.withColumn("timestamp",to_timestamp("datetime", 'MM/dd/yyyy HH:mm:ss')).show(truncate=False)
#df.select(to_timestamp(df.datetime, 'MM/dd/yyyy HH:mm:ss').alias('dt')).collect()

+----------------+-------+--------+------+---+-------------------+
|        datetime|    Lat|     Lon|  Base| id|          timestamp|
+----------------+-------+--------+------+---+-------------------+
|9/1/2014 0:01:00|40.2201|-74.0021|B02512|  0|2014-09-01 00:01:00|
|9/1/2014 0:01:00|  40.75|-74.0027|B02512|  1|2014-09-01 00:01:00|
|9/1/2014 0:03:00|40.7559|-73.9864|B02512|  2|2014-09-01 00:03:00|
|9/1/2014 0:06:00| 40.745|-73.9889|B02512|  3|2014-09-01 00:06:00|
|9/1/2014 0:11:00|40.8145|-73.9444|B02512|  4|2014-09-01 00:11:00|
|9/1/2014 0:12:00|40.6735|-73.9918|B02512|  5|2014-09-01 00:12:00|
|9/1/2014 0:15:00|40.7471|-73.6472|B02512|  6|2014-09-01 00:15:00|
|9/1/2014 0:16:00|40.6613|-74.2691|B02512|  7|2014-09-01 00:16:00|
|9/1/2014 0:32:00|40.3745|-73.9999|B02512|  8|2014-09-01 00:32:00|
|9/1/2014 0:33:00|40.7633|-73.9773|B02512|  9|2014-09-01 00:33:00|
|9/1/2014 0:33:00|40.7467|-73.6131|B02512| 10|2014-09-01 00:33:00|
|9/1/2014 0:37:00|40.8105|  -73.96|B02512| 11|2014-09-01 00:37

In [5]:

df2=(df2
  .withColumn('yearday', F.dayofyear(F.col("timestamp")))
 .withColumn('Month', F.month(F.col('timestamp')))
 .withColumn('dayofweek', F.dayofweek(F.col('timestamp')))
 .withColumn('YEAR', F.year(F.col('timestamp')))
 .withColumn('QUARTER', F.quarter(F.col('timestamp')))
 .withColumn('Month', F.month(F.col('timestamp')))
 .withColumn('WeekOfYear', F.weekofyear(F.col('timestamp')))
 .withColumn('Week', F.date_trunc('week',F.col('timestamp')))
)

df2.show()

+----------------+-------+--------+------+---+-------------------+-------+-----+---------+----+-------+----------+-------------------+
|        datetime|    Lat|     Lon|  Base| id|          timestamp|yearday|Month|dayofweek|YEAR|QUARTER|WeekOfYear|               Week|
+----------------+-------+--------+------+---+-------------------+-------+-----+---------+----+-------+----------+-------------------+
|9/1/2014 0:01:00|40.2201|-74.0021|B02512|  0|2014-09-01 00:01:00|    244|    9|        2|2014|      3|        36|2014-09-01 00:00:00|
|9/1/2014 0:01:00|  40.75|-74.0027|B02512|  1|2014-09-01 00:01:00|    244|    9|        2|2014|      3|        36|2014-09-01 00:00:00|
|9/1/2014 0:03:00|40.7559|-73.9864|B02512|  2|2014-09-01 00:03:00|    244|    9|        2|2014|      3|        36|2014-09-01 00:00:00|
|9/1/2014 0:06:00| 40.745|-73.9889|B02512|  3|2014-09-01 00:06:00|    244|    9|        2|2014|      3|        36|2014-09-01 00:00:00|
|9/1/2014 0:11:00|40.8145|-73.9444|B02512|  4|2014-09-0

In [6]:
df2=(df2
  .withColumn('dayofmonth', F.dayofmonth(F.col("timestamp")))
 .withColumn('hour', F.hour(F.col('timestamp')))
 )
df2.show()

+----------------+-------+--------+------+---+-------------------+-------+-----+---------+----+-------+----------+-------------------+----------+----+
|        datetime|    Lat|     Lon|  Base| id|          timestamp|yearday|Month|dayofweek|YEAR|QUARTER|WeekOfYear|               Week|dayofmonth|hour|
+----------------+-------+--------+------+---+-------------------+-------+-----+---------+----+-------+----------+-------------------+----------+----+
|9/1/2014 0:01:00|40.2201|-74.0021|B02512|  0|2014-09-01 00:01:00|    244|    9|        2|2014|      3|        36|2014-09-01 00:00:00|         1|   0|
|9/1/2014 0:01:00|  40.75|-74.0027|B02512|  1|2014-09-01 00:01:00|    244|    9|        2|2014|      3|        36|2014-09-01 00:00:00|         1|   0|
|9/1/2014 0:03:00|40.7559|-73.9864|B02512|  2|2014-09-01 00:03:00|    244|    9|        2|2014|      3|        36|2014-09-01 00:00:00|         1|   0|
|9/1/2014 0:06:00| 40.745|-73.9889|B02512|  3|2014-09-01 00:06:00|    244|    9|        2|2014

In [7]:
from pyspark.sql.functions import count, avg
df2.drop('datetime')
df2.drop('Week')
all_fetures_df=df2.groupBy("YEAR", "QUARTER","Month","Week","WeekOfYear","yearday" ,"dayofmonth","dayofweek","hour").agg(count("id"))
all_fetures_df.show()

+----+-------+-----+-------------------+----------+-------+----------+---------+----+---------+
|YEAR|QUARTER|Month|               Week|WeekOfYear|yearday|dayofmonth|dayofweek|hour|count(id)|
+----+-------+-----+-------------------+----------+-------+----------+---------+----+---------+
|2014|      3|    9|2014-09-01 00:00:00|        36|    244|         1|        2|   5|      283|
|2014|      3|    9|2014-09-01 00:00:00|        36|    246|         3|        4|  10|     1232|
|2014|      3|    9|2014-09-08 00:00:00|        37|    251|         8|        2|  22|     1117|
|2014|      3|    9|2014-09-22 00:00:00|        39|    265|        22|        2|   6|     1357|
|2014|      3|    8|2014-08-04 00:00:00|        32|    217|         5|        3|  13|     1166|
|2014|      3|    8|2014-08-04 00:00:00|        32|    222|        10|        1|  20|     1155|
|2014|      3|    8|2014-08-11 00:00:00|        33|    224|        12|        3|   6|      987|
|2014|      3|    8|2014-08-11 00:00:00|

In [8]:
from pyspark.sql.types import IntegerType

all_fetures_df.printSchema()

#all_fetures_df=all_fetures_df.withColumn("YEAR",all_fetures_df.YEAR.cast(IntegerType()))
#all_fetures_df=all_fetures_df.withColumn("QUARTER",all_fetures_df.QUARTER.cast(IntegerType()))
#all_fetures_df=all_fetures_df.withColumn("Month",all_fetures_df.Month.cast(IntegerType()))
#all_fetures_df=all_fetures_df.withColumn("Week",all_fetures_df.Week.cast(IntegerType()))
#all_fetures_df=all_fetures_df.withColumn("WeekOfYear",all_fetures_df.WeekOfYear.cast(IntegerType()))
#all_fetures_df=all_fetures_df.withColumn("yearday",all_fetures_df.yearday.cast(IntegerType()))
#all_fetures_df=all_fetures_df.withColumn("dayofmonth",all_fetures_df.dayofmonth.cast(IntegerType()))
#all_fetures_df=all_fetures_df.withColumn("dayofweek",all_fetures_df.dayofweek.cast(IntegerType()))
#all_fetures_df=all_fetures_df.withColumn("hour",all_fetures_df.hour.cast(IntegerType()))

all_fetures_df=all_fetures_df.sort(["YEAR", "QUARTER","Month","WeekOfYear","yearday", "dayofmonth","dayofweek","hour"],ascending=True)
all_fetures_df.show()

root
 |-- YEAR: integer (nullable = true)
 |-- QUARTER: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Week: timestamp (nullable = true)
 |-- WeekOfYear: integer (nullable = true)
 |-- yearday: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- count(id): long (nullable = false)

+----+-------+-----+-------------------+----------+-------+----------+---------+----+---------+
|YEAR|QUARTER|Month|               Week|WeekOfYear|yearday|dayofmonth|dayofweek|hour|count(id)|
+----+-------+-----+-------------------+----------+-------+----------+---------+----+---------+
|2014|      2|    4|2014-03-31 00:00:00|        14|     91|         1|        3|   0|      138|
|2014|      2|    4|2014-03-31 00:00:00|        14|     91|         1|        3|   1|       66|
|2014|      2|    4|2014-03-31 00:00:00|        14|     91|         1|        3|   2|       53|
|2014|      2|    4

In [9]:
from pyspark.ml.feature import VectorAssembler
all_features=['YEAR','QUARTER','Month','WeekOfYear','yearday','dayofmonth','dayofweek','hour']

In [10]:
from pyspark.ml.feature import VectorAssembler
all_features=['YEAR','QUARTER','Month','WeekOfYear','yearday','dayofmonth','dayofweek','hour']
all_features_assembler = VectorAssembler(inputCols=all_features, outputCol='AllFeatures')
af_transformed_dataset = all_features_assembler.transform(all_fetures_df)
af_transformed_dataset.show(truncate=False)

+----+-------+-----+-------------------+----------+-------+----------+---------+----+---------+---------------------------------------+
|YEAR|QUARTER|Month|Week               |WeekOfYear|yearday|dayofmonth|dayofweek|hour|count(id)|AllFeatures                            |
+----+-------+-----+-------------------+----------+-------+----------+---------+----+---------+---------------------------------------+
|2014|2      |4    |2014-03-31 00:00:00|14        |91     |1         |3        |0   |138      |[2014.0,2.0,4.0,14.0,91.0,1.0,3.0,0.0] |
|2014|2      |4    |2014-03-31 00:00:00|14        |91     |1         |3        |1   |66       |[2014.0,2.0,4.0,14.0,91.0,1.0,3.0,1.0] |
|2014|2      |4    |2014-03-31 00:00:00|14        |91     |1         |3        |2   |53       |[2014.0,2.0,4.0,14.0,91.0,1.0,3.0,2.0] |
|2014|2      |4    |2014-03-31 00:00:00|14        |91     |1         |3        |3   |93       |[2014.0,2.0,4.0,14.0,91.0,1.0,3.0,3.0] |
|2014|2      |4    |2014-03-31 00:00:00|14      

In [ ]:
af_transformed_dataset.select('YEAR').distinct().collect()

[Row(YEAR=2014)]

In [ ]:
af_transformed_dataset.select('Month').distinct().collect()

[Row(Month=6),
 Row(Month=5),
 Row(Month=9),
 Row(Month=4),
 Row(Month=8),
 Row(Month=7)]

In [ ]:
af_transformed_dataset.groupBy("Month").agg(count("yearday")).show()


+-----+--------------+
|Month|count(yearday)|
+-----+--------------+
|    6|           720|
|    5|           744|
|    9|           719|
|    4|           720|
|    8|           744|
|    7|           744|
+-----+--------------+



In [11]:
af_transformed_dataset=af_transformed_dataset.withColumnRenamed("count(id)","label")
af_train_dataset = af_transformed_dataset.filter(F.col('Month')!=9)
af_test_dataset = af_transformed_dataset.filter(F.col('Month')==9)

In [13]:
from pyspark.ml.classification import LogisticRegression

print('Training Dataset Count: ' , af_train_dataset.count())
print('Training Dataset Count: ' , af_test_dataset.count())

logreg1=LogisticRegression(featuresCol='AllFeatures',labelCol='label',maxIter=10)
learningModel1=logreg1.fit(af_train_dataset)
Predictions1=learningModel1.transform(af_test_dataset)
Predictions1.show()

Training Dataset Count:  3672
Training Dataset Count:  719
+----+-------+-----+-------------------+----------+-------+----------+---------+----+-----+--------------------+--------------------+--------------------+----------+
|YEAR|QUARTER|Month|               Week|WeekOfYear|yearday|dayofmonth|dayofweek|hour|label|         AllFeatures|       rawPrediction|         probability|prediction|
+----+-------+-----+-------------------+----------+-------+----------+---------+----+-----+--------------------+--------------------+--------------------+----------+
|2014|      3|    9|2014-09-01 00:00:00|        36|    244|         1|        2|   0|  699|[2014.0,3.0,9.0,3...|[-1.0317642486842...|[8.10377653740482...|    1109.0|
|2014|      3|    9|2014-09-01 00:00:00|        36|    244|         1|        2|   1|  490|[2014.0,3.0,9.0,3...|[-1.0334397809969...|[1.08001839949463...|    1109.0|
|2014|      3|    9|2014-09-01 00:00:00|        36|    244|         1|        2|   2|  363|[2014.0,3.0,9.0,3...

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
logreg_evaluator1 = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
logreg_evaluator1.evaluate(Predictions1)
#print("MSE: ", learningModel1.summary.meanSquaredError)
#print("MAE: ", learningModel1.summary.meanAbsoluteError)
print("weightedFMeasure: ", learningModel1.summary.weightedFMeasure)

weightedFMeasure:  <bound method _ClassificationSummary.weightedFMeasure of <pyspark.ml.classification.LogisticRegressionTrainingSummary object at 0x7ffa64285cd0>>


In [14]:
mae=(Predictions1.select(abs(Predictions1['prediction'] - Predictions1['label']))).agg(avg(col("abs((prediction - label))"))).show()
print("MAE: ",mae)

+------------------------------+
|avg(abs((prediction - label)))|
+------------------------------+
|             676.5368567454798|
+------------------------------+

MAE:  None


In [10]:
from pyspark.ml.feature import OneHotEncoder
#encoded_af_dataset=encoded_af_dataset.drop(outCol)
encoded_af_dataset = OneHotEncoder(inputCol='YEAR',outputCol='YEAREncoded',handleInvalid='keep').fit(all_fetures_df).transform(all_fetures_df)
encoded_af_dataset = OneHotEncoder(inputCol='QUARTER',outputCol='QUARTEREncoded',handleInvalid='keep').fit(encoded_af_dataset).transform(encoded_af_dataset)
encoded_af_dataset = OneHotEncoder(inputCol='Month',outputCol='MonthEncoded',handleInvalid='keep').fit(encoded_af_dataset).transform(encoded_af_dataset)
encoded_af_dataset = OneHotEncoder(inputCol='WeekOfYear',outputCol='WeekOfYearEncoded',handleInvalid='keep').fit(encoded_af_dataset).transform(encoded_af_dataset)
encoded_af_dataset = OneHotEncoder(inputCol='yearday',outputCol='yeardayEncoded',handleInvalid='keep').fit(encoded_af_dataset).transform(encoded_af_dataset)
encoded_af_dataset = OneHotEncoder(inputCol='dayofmonth',outputCol='dayofmonthEncoded',handleInvalid='keep').fit(encoded_af_dataset).transform(encoded_af_dataset)
encoded_af_dataset = OneHotEncoder(inputCol='dayofweek',outputCol='dayofweekEncoded',handleInvalid='keep').fit(encoded_af_dataset).transform(encoded_af_dataset)
encoded_af_dataset = OneHotEncoder(inputCol='hour',outputCol='hourEncoded',handleInvalid='keep').fit(encoded_af_dataset).transform(encoded_af_dataset)

all_features=['YEAR','QUARTER','Month','WeekOfYear','yearday','dayofmonth','dayofweek','hour']

encoded_af_dataset.show()

+----+-------+-----+-------------------+----------+-------+----------+---------+----+---------+-------------------+--------------+--------------+-----------------+----------------+-----------------+----------------+---------------+
|YEAR|QUARTER|Month|               Week|WeekOfYear|yearday|dayofmonth|dayofweek|hour|count(id)|        YEAREncoded|QUARTEREncoded|  MonthEncoded|WeekOfYearEncoded|  yeardayEncoded|dayofmonthEncoded|dayofweekEncoded|    hourEncoded|
+----+-------+-----+-------------------+----------+-------+----------+---------+----+---------+-------------------+--------------+--------------+-----------------+----------------+-----------------+----------------+---------------+
|2014|      2|    4|2014-03-31 00:00:00|        14|     91|         1|        3|   0|      138|(2015,[2014],[1.0])| (4,[2],[1.0])|(10,[4],[1.0])|  (41,[14],[1.0])|(274,[91],[1.0])|   (32,[1],[1.0])|   (8,[3],[1.0])| (24,[0],[1.0])|
|2014|      2|    4|2014-03-31 00:00:00|        14|     91|         1|  

In [11]:
for col in all_features:
  encoded_af_dataset=encoded_af_dataset.drop(col)
encoded_af_dataset=encoded_af_dataset.drop("Week")

In [12]:
all_efeatures=['YEAREncoded','QUARTEREncoded','MonthEncoded','WeekOfYearEncoded','yeardayEncoded','dayofmonthEncoded','dayofweekEncoded','hourEncoded']
all_efeatures_assembler = VectorAssembler(inputCols=all_efeatures, outputCol='AllEFeatures')
eaf_transformed_dataset = all_efeatures_assembler.transform(encoded_af_dataset)
eaf_transformed_dataset.show(truncate=False)

+---------+-------------------+--------------+--------------+-----------------+----------------+-----------------+----------------+---------------+----------------------------------------------------------------------------------+
|count(id)|YEAREncoded        |QUARTEREncoded|MonthEncoded  |WeekOfYearEncoded|yeardayEncoded  |dayofmonthEncoded|dayofweekEncoded|hourEncoded    |AllEFeatures                                                                      |
+---------+-------------------+--------------+--------------+-----------------+----------------+-----------------+----------------+---------------+----------------------------------------------------------------------------------+
|138      |(2015,[2014],[1.0])|(4,[2],[1.0]) |(10,[4],[1.0])|(41,[14],[1.0])  |(274,[91],[1.0])|(32,[1],[1.0])   |(8,[3],[1.0])   |(24,[0],[1.0]) |(2408,[2014,2017,2023,2043,2161,2345,2379,2384],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|66       |(2015,[2014],[1.0])|(4,[2],[1.0]) |(10,[4],[1.0])|(41,[14],[1.0])

In [13]:
eaf_transformed_dataset=eaf_transformed_dataset.withColumnRenamed("count(id)","label")
eaf_train_dataset = eaf_transformed_dataset.filter(F.col('Month')!=9)
eaf_test_dataset = eaf_transformed_dataset.filter(F.col('Month')==9)
eaf_train_dataset.show()
eaf_test_dataset.show()

+-----+-------------------+--------------+--------------+-----------------+----------------+-----------------+----------------+---------------+--------------------+
|label|        YEAREncoded|QUARTEREncoded|  MonthEncoded|WeekOfYearEncoded|  yeardayEncoded|dayofmonthEncoded|dayofweekEncoded|    hourEncoded|        AllEFeatures|
+-----+-------------------+--------------+--------------+-----------------+----------------+-----------------+----------------+---------------+--------------------+
|  138|(2015,[2014],[1.0])| (4,[2],[1.0])|(10,[4],[1.0])|  (41,[14],[1.0])|(274,[91],[1.0])|   (32,[1],[1.0])|   (8,[3],[1.0])| (24,[0],[1.0])|(2408,[2014,2017,...|
|   66|(2015,[2014],[1.0])| (4,[2],[1.0])|(10,[4],[1.0])|  (41,[14],[1.0])|(274,[91],[1.0])|   (32,[1],[1.0])|   (8,[3],[1.0])| (24,[1],[1.0])|(2408,[2014,2017,...|
|   53|(2015,[2014],[1.0])| (4,[2],[1.0])|(10,[4],[1.0])|  (41,[14],[1.0])|(274,[91],[1.0])|   (32,[1],[1.0])|   (8,[3],[1.0])| (24,[2],[1.0])|(2408,[2014,2017,...|
|   93|(20

In [14]:
print('Training Dataset Count: ' , eaf_train_dataset.count())
print('Training Dataset Count: ' , eaf_test_dataset.count())

logreg2=LogisticRegression(featuresCol='AllEFeatures',labelCol='label',maxIter=10)
learningModel2=logreg2.fit(eaf_train_dataset)
Predictions2=learningModel2.transform(eaf_test_dataset)
Predictions2.show()

Training Dataset Count:  3672
Training Dataset Count:  719


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 44522)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-dfac75b90c5b>", line 5, in <module>
    learningModel2=logreg2.fit(eaf_train_dataset)
  File "/usr/local/lib/python3.7/dist-packages/pyspark/ml/base.py", line 205, in fit
    return self._fit(dataset)
  File "/usr/local/lib/python3.7/dist-packages/pyspark/ml/wrapper.py", line 379, in _fit
    java_model = self._fit_java(dataset)
  File "/usr/local/lib/python3.7/dist-packages/pyspark/ml/wrapper.py", line 376, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    answer, self.gateway_client, self.target_id, self.name)

ConnectionRefusedError: ignored